In [1]:
import os
import argparse
import glob
import time
import random
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import higher
from torch.utils.tensorboard import SummaryWriter

In [2]:
import utils

from train_2 import EditTrainer
from config import EditConfig


In [4]:
model, tokenizer = utils.loadOTSModel()
dataloader = utils.retrieveDataloader(
        tokenizer, 
        bs=1, 
        dataset='train'
    )
del model

In [5]:
class EditConfig:
    def __init__(self):
        self.inner_lr = 1e-3
        self.outer_lr = 1e-5
        self.epochs = 1
        self.max_training_samps = 2e4
        self.n_edit_steps = 1
        self.cedit = 1
        self.cloc = 1

        self.debug = True
        self.model_save_pt = 2000
        self.model_dir = '../models'

## Test Run

In [ ]:
%load_ext autoreload
%autoreload 2
import utils

from train_2 import EditTrainer
from config import EditConfig


## Testing Language Generation

In [12]:
# text = "Poppies and lupines are the primary flowers that dominate the green, lush rolling meadows in the spring to early summer, but in the late summer the hills turn gold and are covered with flowers such as the cluster-lily, gumweed, Mules ear, and farewell to spring. In shaded areas, one can expect to see thimbleberry and a variety of ferns growing. "

In [13]:
text = "Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made it particularly popular as a means of adding programmable interfaces to existing applications. Van Rossum's vision of a small core language with a large standard library and easily extensible interpreter stemmed from his frustrations with ABC, which espoused the opposite approach."

In [14]:
token_list = tokenizer.encode(text)
token_out = tokenizer(
    text
)
tokens, mask = map(
    torch.tensor, 
    [token_out['input_ids'], token_out['attention_mask']]
)

In [15]:
tokens.size()

torch.Size([79])

In [16]:
feed = tokens[:40]
print(tokenizer.decode(token_list[:40]))

Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made it particularly popular as a means of adding programmable


In [17]:

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
generation_output

GreedySearchDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220, 17479,    13,
           314,  1842,   683,   523,   881,    13,   314,  1842,   683,   523]]), scores=(tensor([[-43.4955, -46.0844, -46.9108,  ..., -57.7377, -51.5319, -48.3616]]), tensor([[-48.8799, -50.3510, -55.1450,  ..., -64.8533, -62.6215, -52.4255]]), tensor([[-67.7690, -65.9575, -67.5762,  ..., -76.5747, -74.8564, -60.7996]]), tensor([[-86.5862, -85.9131, -89.9283,  ..., -97.0860, -86.9367, -87.2296]]), tensor([[-66.0425, -67.5343, -71.4601,  ..., -75.1458, -72.8039, -67.9962]]), tensor([[-59.0425, -63.4027, -69.0216,  ..., -77.3129, -73.8313, -64.7133]]), tensor([[-60.0665, -62.8150, -68.4672,  ..., -72.5271, -70.3083, -63.8665]]), tensor([[-47.8266, -52.4544, -58.7991,  ..., -65.4726, -64.2974, -52.0334]]), tensor([[-71.7299, -70.1800, -72.2610,  ..., -82.0179, -80.4628, -63.5811]]), tensor([[-86.2451, -85.7662, -90.2981,  ..., -97.1315, -86.9278, -86.6646]]), tensor([[-6

In [19]:
tokenizer.decode(tokenizer.eos_token_id)

'<|endoftext|>'

In [20]:
text = "Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made"

In [22]:
prompt_text = text
encoded_prompt = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
input_ids = encoded_prompt

output_sequences = model.generate(
    input_ids=input_ids,
    max_length=20 + len(encoded_prompt[0]),
    temperature=1.2,
#     top_k=args.k,
#     top_p=args.p,
    repetition_penalty=1,
    do_sample=True,
    num_return_sequences=1,
)

# Remove the batch dimension when returning multiple sequences
if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

generated_sequences = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find('<|endoftext|>')]

    # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
    )

    generated_sequences.append(total_sequence)
    print(total_sequence)

generated_sequences

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GENERATED SEQUENCE 1 ===
Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made much of python less complex but it is also able to do other things just as easy for anyone t


['Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made much of python less complex but it is also able to do other things just as easy for anyone t']

In [28]:
prompt_text = text
encoded_prompt = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
input_ids = encoded_prompt

output_sequences = model.generate(
    input_ids=input_ids,
    max_length=20 + len(encoded_prompt[0]),
    temperature=1.2,
#     top_k=args.k,
#     top_p=args.p,
    repetition_penalty=1,
    do_sample=True,
    num_return_sequences=5,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [36]:
print(input_ids.size())
for i in range(4):
    print(output_sequences[i, 49:])

torch.Size([1, 50])
tensor([  256, 34369,    13,  1320,   531,   532,   314,   836,   470,   892,
         9599,   508, 14759, 11361,  1865,   815,   307,   379,   477, 32064,
           13])
tensor([ 256, 7834,  306,  284, 3551, 2438,  357,   72,   13,   68,   13, 3551,
        6725,  326,  314, 1101, 1262,  287,  257, 2248,  737])
tensor([ 256, 5430, 1223,  656,  663, 2779,  393,  655, 1804, 1243, 1088,  340,
          13,  198,  198,  198, 3844,   11,  981,  428, 1244])
tensor([  256,  7834,   278,   351, 11361,    13,   198,   198,   198,  1135,
          550,  2904,  3066,   422,   262,  3726,   326,   356,   655,  2622,
          284])


In [25]:
import random

class SelfSampleTrainer(EditTrainer):
    def __init__(self, config, dataloader, model_path=None):
        super().__init__(config, dataloader, model_path) 
        
        self.finetuned = utils.loadTrainedModel(
            "../models/finetune/gpt2_epoch0_ts10000.20210310.18.03.1615401990", 
            tokenizer=False
        )
        self.finetuned.to(self.device)
        
    def genModelText(self, lm_tokens, edit_locs):
        

        input_ids = lm_tokens[:, :edit_locs.min()]
        input_size = input_ids.size()[-1]
        
        self.model.eval()
        print("generating")
        output_sequence = self.finetuned.generate(
            input_ids=input_ids,
            max_length=input_size + 15,
            temperature=1.2,
            repetition_penalty=1.0,
            do_sample=True,
            num_return_sequences=10,
        )

        edit_tokens = random.choice(output_sequence).unsqueeze(0)
        edit_mask = torch.ones(edit_tokens.shape, dtype=torch.long)
        edit_labels = torch.zeros(edit_tokens.shape, dtype=torch.long) - 100
        edit_labels[:, input_size:] = edit_tokens[:, input_size:]
        edit_labels = edit_labels.to(self.device)
        edit_tokens, edit_mask = edit_tokens.to(self.device), edit_mask.to(self.device)

        return edit_tokens, edit_mask, edit_labels

    
    def run(self):

        if not self.config.debug:
            torch.save(self.config, self.hyperspath)

        self.model.train()
        self.model.to(self.device)
        opt = torch.optim.Adam(
            self.model.parameters(), 
            self.config.outer_lr
            )
        
        global_iter = 0
        print("Starting Training")

        for epoch in range(self.config.epochs):
            self.epoch = epoch
            
            for train_step, (lm_data, edit_example, ent) in enumerate(self.data):
            
                lm_tokens, lm_mask = lm_data
                lm_tokens, lm_mask = lm_tokens.to(self.device), lm_mask.to(self.device)
                lm_labels = lm_tokens.masked_fill(lm_mask == 0, -100)
                
                ent_tokens = ent[0].flatten()
                ent_tokens = ent_tokens[ent_tokens != 50256]
                edit_locs = utils.locateEntityEdit(edit_example[0], ent_tokens)
                if edit_locs.size == 0 or edit_locs.min() == 0:
                    continue
                
                try:
                    edit_tokens, edit_mask, edit_labels = self.genModelText(lm_tokens, edit_locs)
                except RuntimeError:
                    breakpoint()
                
                
                inner_opt = torch.optim.SGD(
                    self.model.transformer.h[-3:].parameters(), 
                    lr=self.config.inner_lr
                    )

                with higher.innerloop_ctx(
                    self.model, 
                    inner_opt, 
                    copy_initial_weights=False, 
                    track_higher_grads=True
                    ) as (fmodel, diffopt):
                    
                    for edit_step in range(self.config.n_edit_steps):

                        loss = fmodel(
                            edit_tokens, 
                            attention_mask=edit_mask,
                            labels=edit_labels
                        ).loss
                        diffopt.step(loss)

                    edit_out = fmodel(
                        edit_tokens, 
                        attention_mask=edit_mask,
                        labels=edit_labels
                    )
                    l_edit = edit_out.loss
                    
                    base_out = self.model(
                        lm_tokens, 
                        attention_mask=lm_mask,
                        labels=lm_labels
                    )
                    l_base = base_out.loss

                    edited_base_out = fmodel(
                        lm_tokens, 
                        attention_mask=lm_mask,
                        labels=lm_labels
                    )

                    l_loc =  (
                        F.softmax(base_out.logits.detach(), dim=-1) *
                        (
                            F.log_softmax(base_out.logits.detach(), dim=-1) - 
                            F.log_softmax(edited_base_out.logits, dim=-1)
                        )).sum(-1).mean()
                    
                    total_loss = (
                        l_base + 
                        self.config.cloc * l_loc  + 
                        self.config.cedit * l_edit
                        )
                    total_loss.backward()

                    # accumulate grads 
                    if train_step % 5 == 0:
                        opt.step()
                        opt.zero_grad()
                    
                    global_iter += 1
                    
                    loss_dict = {
                        "l_base": l_base, "l_edit": l_edit, 
                        "l_loc": l_loc, "total": total_loss
                        }
                    self.echo(train_step, **loss_dict)
                    if not self.config.debug:
                        self.tensorBoard(global_iter, **loss_dict)
                        self.saveModel(self.model, train_step)

                        if train_step % 1000 == 0:
                            self.validateEditTraining()

        if not self.config.debug:
            self.saveModel(self.model, train_step)
        self.writer.flush()

In [26]:
trainer = SelfSampleTrainer(EditConfig(), dataloader)
trainer.run()

Starting Training


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generating


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


('Epoch: 0; TrainStep 0; ', 'l_base 4.847848892211914; l_edit 4.031960487365723; l_loc 9.952959953807294e-05; total 8.87990951538086')
generating


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


('Epoch: 0; TrainStep 77; ', 'l_base 3.7875313758850098; l_edit 4.1634321212768555; l_loc 0.00019793372484855354; total 7.9511613845825195')
generating


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


('Epoch: 0; TrainStep 78; ', 'l_base 3.467377185821533; l_edit 4.028688907623291; l_loc 0.00019779133435804397; total 7.4962639808654785')
generating


KeyboardInterrupt: 